In [4]:
import pandas as pd
import numpy as np
import requests
# from api_key import key
from datetime import date
from dateutil.relativedelta import relativedelta

In [5]:
def get_stock_data(ticker, multiplier, timespan, from_date, to_date):
    # Make the API request
    api_url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/{multiplier}/{timespan}/{from_date}/{to_date}"
    response = requests.get(api_url, params={"apiKey": "wQ5FjyMjpTSO2j5vBxbLuIp72hwYd5E5"})

    # Check for errors
    if response.status_code != 200:
        raise Exception(f"Error {response.status_code}: {response.text}")

    # Extract the data from the response
    data = response.json()["results"]
    stock_data = []
    for d in data:
        stock_data.append({
            "Date": pd.to_datetime(d["t"], unit='ms').date(),
            "Open": d["o"],
            "High": d["h"],
            "Low": d["l"],
            "Close": d["c"],
            "Adj Close": d["c"],
            "Volume": d["v"],
        })
     
     # Convert the data to a dataframe
    df = pd.DataFrame(stock_data)
    df = df.reset_index(drop=True)
    
    return df

In [6]:
data = get_stock_data('AAPL', 1, 'day', '2021-02-28', '2023-02-28')

data.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-03-26,120.35,121.4800,118.9200,121.210,121.210,94071234.0
1,2021-03-29,121.65,122.5800,120.7299,121.390,121.390,80819203.0
2,2021-03-30,120.11,120.4031,118.8600,119.900,119.900,84507919.0
3,2021-03-31,121.65,123.5200,121.1500,122.150,122.150,118323826.0
4,2021-04-01,123.66,124.1800,122.4900,123.000,123.000,75089134.0
5,2021-04-05,123.87,126.1601,123.0700,125.900,125.900,88251175.0
6,2021-04-06,126.50,127.1300,125.6500,126.210,126.210,80171253.0
7,2021-04-07,125.83,127.9200,125.1400,127.900,127.900,83256716.0
8,2021-04-08,128.95,130.3900,128.5200,130.360,130.360,88844591.0
9,2021-04-09,129.80,133.0400,129.4700,132.995,132.995,106615653.0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485 entries, 0 to 484
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       485 non-null    object 
 1   Open       485 non-null    float64
 2   High       485 non-null    float64
 3   Low        485 non-null    float64
 4   Close      485 non-null    float64
 5   Adj Close  485 non-null    float64
 6   Volume     485 non-null    float64
dtypes: float64(6), object(1)
memory usage: 26.6+ KB


In [8]:
split = int(round(data.size*0.7,0))
train_size = split
test_size = data.size - split

data_train = data[:split]
data_test = data[split:]

In [16]:
def subsample_sequence(sequence, length, horizon):

    # Bounds of sampling
    last_possible = len(sequence) - length - horizon
    random_start = np.random.randint(0, last_possible)

    # Sample
    X = sequence[random_start:random_start +length].drop(columns=['Open','Date', 'Adj Close']) # Remove target closing price & date
    y = sequence.loc[random_start+length+horizon-1]['Close'] # Target is closing price

    return X, y


subsample_sequence(data, 3, 1)

(       High     Low   Close      Volume
 311  137.06  133.32  135.87  81000488.0
 312  137.76  133.91  135.35  73409234.0
 313  138.59  135.63  138.27  72433768.0,
 141.66)

In [10]:
def get_X_y(sequence, length, horizon, number_of_samples):
    
    X, y = [], []
    
    # Do as many samples as specified
    for sample in range(1,number_of_samples + 1):
        
        # Record sample X & y
        xi, yi = subsample_sequence(sequence, length, horizon)
        X.append(np.array(xi.values.T.tolist()).T) # Getting the right shape (sequences, observations,features)
        y.append(yi)

    return np.array(X), np.array(y)

In [18]:
X_train, y_train = get_X_y(sequence=data_train,
               length=100,
               horizon=1,
               number_of_samples=10000)

In [19]:
# 10000 subsamples, each of lenght 100 , each containg 4 features
X_train.shape

(10000, 100, 4)

In [20]:
y_train = y_train.reshape(10000,1)

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers

def init_model():

    model = models.Sequential()

    model.add(layers.LSTM(15, return_sequences=True, activation='tanh', input_shape=(100,4)))
    model.add(layers.LSTM(20, return_sequences=True, activation='tanh'))
    model.add(layers.LSTM(15, return_sequences=True, activation='tanh'))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='linear'))

    model.compile(loss='mse',
                  optimizer='rmsprop',
                  metrics=['mae'])


    return model

init_model().summary()

In [49]:
from tensorflow.keras.callbacks import EarlyStopping
model = init_model()

es = EarlyStopping(patience=20)

model.fit(X_train, y_train, 
          epochs=8, 
          batch_size=16, 
          verbose=1, 
          callbacks = [es],
          validation_split=0.2)

Epoch 1/8
500/500 [==============================] - 13s 22ms/step - loss: 18263.5371 - mae: 134.0073 - val_loss: 12576.9932 - val_mae: 111.4100
Epoch 2/8
500/500 [==============================] - 11s 22ms/step - loss: 6928.1128 - mae: 79.7247 - val_loss: 1966.5253 - val_mae: 42.3846
Epoch 3/8
500/500 [==============================] - 11s 22ms/step - loss: 482.6674 - mae: 17.1270 - val_loss: 175.7111 - val_mae: 10.8601
Epoch 4/8
500/500 [==============================] - 11s 22ms/step - loss: 177.7767 - mae: 10.9767 - val_loss: 176.2204 - val_mae: 10.9739
Epoch 5/8
500/500 [==============================] - 10s 21ms/step - loss: 178.0064 - mae: 10.9976 - val_loss: 175.7184 - val_mae: 10.8806
Epoch 6/8
500/500 [==============================] - 10s 21ms/step - loss: 177.8316 - mae: 10.9821 - val_loss: 175.6906 - val_mae: 10.8192
Epoch 7/8
500/500 [==============================] - 11s 21ms/step - loss: 177.9033 - mae: 10.9842 - val_loss: 175.8321 - val_mae: 10.9193
Epoch 8/8
500/500 [